## Imports

In [ ]:
import json
import os
import sys

import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

from sklearn.svm import SVC, SVR
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import accuracy_score, mean_squared_error, make_scorer
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold, ValidationCurveDisplay
from sklearn.multioutput import MultiOutputRegressor
from sklearn.preprocessing import PolynomialFeatures

from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.initializers import RandomUniform, RandomNormal, HeNormal, GlorotUniform, Constant, Zeros
from keras.optimizers import SGD, Adam
from tensorflow.keras.regularizers import l2


dir_parts = os.getcwd().split(os.path.sep)
root_index = dir_parts.index('ML-B')
root_path = os.path.sep.join(dir_parts[:root_index + 1])
sys.path.append(root_path + '/code/')
from data.data_config import Dataset
from data.data_utils import load_monk, load_cup, store_monk_result, store_cup_result
from hyperparameter_tuning import grid_search, random_search, tuning_search_top_configs
from training.solver import Solver
from training.metrics import mean_euclidean_error as mee

%load_ext autoreload
%autoreload 2

# Ensemble
In this notebook we apply several ensemble techniques to combine the blind test predictions of our (best) chosen model.

Specifically, the ensemble approaches we consider are the following:
- *Arithmetic Average*;
- *Weighted Average*;
- *Stacking*.

## Settings

In [ ]:
MODEL_NAME = 'Ensemble'
VAL_SPLIT = 0.2 # validation split percentage
INTERNAL_TEST_SPLIT = 0.1 # internal test split percentage
RANDOM_STATE = 128 # reproducibility
N_SPLITS = 5 # cross-validation
POLY_DEGREE = 3 # Polynomial features pre-processing

## Path

In [ ]:
# Directories
results_dir = root_path + '/results/' + MODEL_NAME

# Filepaths (MONK)
m1_dev_path, m1_test_path = Dataset.MONK_1.dev_path, Dataset.MONK_1.test_path # MONK 1
m2_dev_path, m2_test_path = Dataset.MONK_2.dev_path, Dataset.MONK_2.test_path # MONK 2
m3_dev_path, m3_test_path = Dataset.MONK_3.dev_path, Dataset.MONK_3.test_path # MONK 3

# Filepaths (CUP)
cup_dev_path, cup_test_path = Dataset.CUP.dev_path, Dataset.CUP.test_path

## Arithmetic Average

In [ ]:
def ensemble_arithmetic_average(nn_sgd_preds, nn_adam_preds, svr_preds):
    """Compute and return the arithmetic average of three models predictions."""
    #return (svr_preds + nn_sgd_preds + nn_adam_preds) / 3
    return np.mean(np.array([nn_adam_preds, nn_sgd_preds, svr_preds]), axis=0)

## Weighted Averange

In [ ]:
# the weight used is (N - mee)
def ensemble_weighted_averange(N: float, svr_preds, nn_sgd_preds, nn_adam_preds, mee_svr: float, mee_nn_sgd: float, mee_nn_adam: float):
    """Compute and return the weighted average of the three models predictions.
    
    Args:
        - N: the maximum (worse) value possible
    """
    
    if (N < mee_svr or N < mee_nn_sgd or N < mee_nn_adam):
        raise ValueError("N must be greater than mee")
        
    w_nom = svr_preds*(N-mee_svr)**2 + nn_sgd_preds*(N-mee_nn_sgd)**2 + nn_adam_preds*(N-mee_nn_adam)**2
    w_den = (N - mee_svr)**2 + (N - mee_nn_sgd)**2 + (N - mee_nn_adam)**2
            
    return w_nom / w_den

# CUP
According to the mean validation MEE achieved during KFold cross-validation, we have selected three models for our ensembles.

The chosen models are:
- *Neural Network with SGD and mini-batch (NN-SGD)*;
- *Neural Network with ADAM and mini-batch (NN-ADAM)*;
- Support Vector Regressor (SVR).

In [ ]:
# Load CUP
x_dev_cup, y_dev_cup, x_test_cup = load_cup(cup_dev_path, cup_test_path)

In [ ]:
@keras.utils.register_keras_serializable()
def mean_euclidean_error(y_true: np.ndarray, y_pred: np.ndarray):
    """
    Utility function to compute the Mean Euclidean Error (MEE) between 
    true and predicted values for a tensorflow model. 
    Return the MEE score as a tensor.

    Required arguments:
    - y_true: array containing true values (ground truth).
    - y_pred: array containing predicted values.
    """
    return tf.reduce_mean(tf.sqrt(tf.reduce_sum(tf.square(y_pred - y_true), axis=-1)))

## Dev - Internal Test Split 
The development dataset is split between training and internal test ($90-10$). Then, to decide whether to take a specific model or ensemble, we further split the train data into train and validation. Thus, the final split of the development data is ($70-20-10$) for training, validation and internal set respectively.

The validation MEE will be used for our final model choice.

In [ ]:
# Split dev data into train - internal test
x_dev_cup_tmp, x_internal_test_cup, y_dev_cup_tmp, y_internal_test_cup = train_test_split(
    x_dev_cup, 
    y_dev_cup, 
    test_size=INTERNAL_TEST_SPLIT, 
    random_state=RANDOM_STATE
)

In [ ]:
# Split dev data into train - internal test
x_train_cup, x_val_cup, y_train_cup, y_val_cup = train_test_split(
    x_dev_cup_tmp, 
    y_dev_cup_tmp, 
    test_size=(VAL_SPLIT / (1 - INTERNAL_TEST_SPLIT)), 
    random_state=RANDOM_STATE
)

## Polynomial features pre-processing
We create a version of our dataset to which PolynoMialFeatures pre-processing is applied with a fixed degree.

In [ ]:
# Polynomial features pre-processing
poly = PolynomialFeatures(degree=POLY_DEGREE)
x_train_cup_poly = poly.fit_transform(x_train_cup)
x_val_cup_poly = poly.transform(x_val_cup)
x_internal_test_cup_poly = poly.transform(x_internal_test_cup)
#x_test_cup = poly.transform(x_test_cup)

## Models' utils

### NN-SGD

In [ ]:
# Best configuration NN-SGD
best_config_sgd = {
    'lr': 0.00035,
    'h_dim': 100,
    'n_layers': 3,
    'activation': 'tanh',
    'reg': 0.001,
    'momentum': 0.95, 
    'batch_size': 32,
}

def get_nn_sgd_regressor(hparams: dict, in_dim: int):
    """Returns a NN with SGD regressor.
    
    Args:
        - hparams: a set of hyper-parameters
        - in_dim: input dimension [1]
    """
    
    if hparams['activation'] == 'tanh':
        initializer = GlorotUniform(seed=RANDOM_STATE) # Glorot (Xavier)
        bias_initializer = Zeros()
    elif hparams['activation'] == 'ReLU':
        initializer = HeNormal(seed=RANDOM_STATE) # He (Kaiming)
        bias_initializer = Constant(0.1)
        
    reg = l2(hparams['reg'])
        
    model = Sequential()
    model.add(Dense(
        hparams['h_dim'], 
        activation=hparams['activation'], 
        input_shape=(in_dim,), 
        kernel_regularizer=l2(hparams['reg']),
        kernel_initializer=initializer,
        bias_initializer=bias_initializer))

    h_dim = hparams['h_dim']
    for i in range(hparams['n_layers'] - 1):
        model.add(
            Dense(
                h_dim, 
                activation=hparams['activation'],
                kernel_regularizer=l2(hparams['reg']),
                kernel_initializer=initializer,
                bias_initializer=bias_initializer))
        h_dim //= 2
        
    model.add(Dense(
        3, 
        activation='linear', 
        kernel_regularizer=l2(hparams['reg']), 
        kernel_initializer=initializer,
        bias_initializer=bias_initializer))

    optimizer = SGD(learning_rate=hparams['lr'], momentum=hparams['momentum'])
    model.compile(optimizer=optimizer, loss='mse', metrics=[mean_euclidean_error])
    return model

### NN-ADAM

In [ ]:
# Best configuration NN-ADAM
best_config_adam = {
    'lr': 0.00026, 
    'n_layers': 3, 
    'h_dim': 128, 
    'activation': 'tanh', 
    'reg': 0.001, 
    'beta_1': 0.87, 
    'beta_2': 0.96, 
    'batch_size': 32,
}

def get_nn_adam_regressor(hparams: dict, in_dim: int):
    """Returns a NN with ADAM regressor.
    
    Args:
        - hparams: a set of hyper-parameters
        - in_dim: input dimension [1]
    """
    
    if hparams['activation'] == 'tanh':
        initializer = GlorotUniform(seed=RANDOM_STATE) # Glorot (Xavier)
        bias_initializer = Zeros()
    elif hparams['activation'] == 'ReLU':
        initializer = HeNormal(seed=RANDOM_STATE) # He (Kaiming)
        bias_initializer = Constant(0.1)
        
    reg = l2(hparams['reg'])
        
    model = Sequential()
    model.add(Dense(
        hparams['h_dim'], 
        activation=hparams['activation'], 
        input_shape=(in_dim,), 
        kernel_regularizer=l2(hparams['reg']),
        kernel_initializer=initializer,
        bias_initializer=bias_initializer))


    h_dim = hparams['h_dim']
    for i in range(hparams['n_layers'] - 1):
        model.add(
            Dense(
                h_dim, 
                activation=hparams['activation'],
                kernel_regularizer=l2(hparams['reg']),
                kernel_initializer=initializer,
                bias_initializer=bias_initializer))
        h_dim //= 2


    model.add(Dense(
        3, 
        activation='linear', 
        kernel_regularizer=l2(hparams['reg']), 
        kernel_initializer=initializer,
        bias_initializer=bias_initializer))
    
    optimizer = Adam(
        learning_rate=hparams['lr'],
        beta_1=hparams['beta_1'], 
        beta_2=hparams['beta_2'])
    model.compile(optimizer=optimizer, loss='mse', metrics=[mean_euclidean_error])
    return model

### SVR

In [ ]:
# Best configuration SVR
best_config_svr = {
    'C': 2000,
    'epsilon': 0.07,
    'kernel': 'rbf',
    'gamma': 'scale',
}

# Training and Validation
Here we leverage the $70-20$ training-validation split to perform model selection w.r.t. the ensemble and the single models. Based on the validation MEE we're going to decide which is our model/ensemble for submitting the blind test predictions.

### NN-SGD

In [ ]:
# Train/Val NN-SGD
model_nn_sgd = get_nn_sgd_regressor(best_config_sgd, x_train_cup.shape[1])
solver = Solver(model_nn_sgd, x_train_cup_poly, y_train_cup, x_internal_test_cup_poly, y_internal_test_cup, target='val_mean_euclidean_error')
solver.train(epochs=800, patience=50)

In [ ]:
nn_sgd_train_preds = model_nn_sgd.predict(x_train_cup_poly)
nn_sgd_val_preds = model_nn_sgd.predict(x_val_cup_poly)
mee_sgd_train = float(mean_euclidean_error(y_train_cup, nn_sgd_train_preds))
mee_sgd_val = float(mean_euclidean_error(y_val_cup, nn_sgd_val_preds))

print(f"TRAIN MEE: {mean_euclidean_error(y_train_cup, nn_sgd_train_preds)} - VAL MEE: {mean_euclidean_error(y_val_cup, nn_sgd_val_preds)}")

### NN-ADAM

In [ ]:
# Train/Val NN-SGD
model_nn_adam = get_nn_sgd_regressor(best_config_sgd, x_train_cup.shape[1])
solver = Solver(model_nn_adam, x_train_cup_poly, y_train_cup, x_internal_test_cup_poly, y_internal_test_cup, target='val_mean_euclidean_error')
solver.train(epochs=1500, patience=100)

In [ ]:
nn_adam_train_preds = model_nn_adam.predict(x_train_cup_poly)
nn_adam_val_preds = model_nn_adam.predict(x_val_cup_poly)
mee_adam_train = float(mean_euclidean_error(y_train_cup, nn_adam_train_preds))
mee_adam_val = float(mean_euclidean_error(y_val_cup, nn_adam_val_preds))

print(f"TRAIN MEE: {mean_euclidean_error(y_train_cup, nn_adam_train_preds)} - VAL MEE: {mean_euclidean_error(y_val_cup, nn_adam_val_preds)}")

### SVR

In [ ]:
# Train SVR
multi_svr = MultiOutputRegressor(SVR(**best_config_svr))
multi_svr.fit(x_train_cup_poly, y_train_cup)

In [ ]:
svr_train_preds = multi_svr.predict(x_train_cup_poly)
svr_val_preds = multi_svr.predict(x_val_cup_poly)
mee_svr_train = mee(y_train_cup, svr_train_preds)
mee_svr_val = mee(y_val_cup, svr_val_preds)

print(f"TRAIN MEE: {mee(y_train_cup, svr_train_preds)} - VAL MEE: {mee(y_val_cup, svr_val_preds)}")

### Ensembles

In [ ]:
# Arithmetic average
ar_ensemble_val_preds = ensemble_arithmetic_average(svr_val_preds, nn_sgd_val_preds, nn_adam_val_preds)
print(f"ARITHMETIC AVERAGE VAL MEE: {mean_euclidean_error(y_val_cup, ar_ensemble_val_preds)}")

In [ ]:
# Weighted average
wa_ensemble_val_preds = ensemble_weighted_averange(
    1, svr_val_preds, nn_sgd_val_preds, 
    nn_adam_val_preds, mee_svr_train, 
    mee_sgd_train, mee_adam_train
)
print(f"WEIGHTED AVERAGE VAL MEE: {mean_euclidean_error(y_val_cup, wa_ensemble_val_preds)}")

In [ ]:
# Ensemble Stacking. Valid estimator: LinearRegression, Ridge or Lasso
#stacking_estimator = LinearRegression()
#stacking_estimator = Lasso(alpha=0.02)
stacking_estimator = Ridge(alpha=30)
x_stack_cup_train = np.hstack((svr_train_preds, nn_sgd_train_preds, nn_adam_train_preds))
x_stack_cup_val = np.hstack((svr_val_preds, nn_sgd_val_preds, nn_adam_val_preds))
stacking_estimator.fit(x_stack_cup_train, y_train_cup)

print(f"RIDGE VAL MEE: {mean_euclidean_error(y_val_cup, stacking_estimator.predict(x_stack_cup_val))}")

In [ ]:
param_name = "alpha"
param_range = np.linspace(0, 80, 100)

ValidationCurveDisplay.from_estimator(estimator, 
                                      x_stack_cup_val, 
                                      y_val_cup, 
                                      param_name=param_name, 
                                      param_range=param_range,
                                      cv=None,
                                      scoring= make_scorer(mee, greater_is_better = False),
                                      negate_score = True,
                                      score_name="MEE",
                                      verbose=1)

plt.show()

# Training and Internal test assessment
Let's perform a re-training of our model on the entire training/validation set. In this way, we're able to leverage the entire training/validation data (early stopping is applied w.r.t. the train MEE). In order to obtain the ensembles' estimates on the (untouched) internal test we load the storeded predictions of the chosen models. Therefore, we're able to perform model assessment and estimate the ensembles' performance on the blind test. Additionally, by using the same predictions used for model assesment of the single models (instead of re-training them) we believe to ensure more meaningful results.

Note that, in this phase, we don't use the internal test in any way (i.e., no training and no validation). We only estimate its errors.

### Load models predictions

In [ ]:
def load_cup_predictions(in_path: str):
    """Utility function to load blind test predictions of a model.
    
    Args:
        - in_dir: the .csv file path
    """
    preds = pd.read_csv(in_path, header=None, delimiter=',', skiprows=4)
    preds.drop(columns=preds.columns[0], axis=1, inplace=True)
    return preds

In [ ]:
# Load train predictions
nn_sgd_trainval_preds = load_cup_predictions(root_path + '/results/NN-SGD/CUP/mean_5_train_preds_poly.csv')
nn_adam_trainval_preds = load_cup_predictions(root_path + '/results/NN-ADAM/CUP/mean_5_train_preds_poly.csv')
svr_trainval_preds = load_cup_predictions(root_path + '/results/SVM/CUP/train_preds_poly.csv')

# Load internal test predictions
nn_sgd_internal_test_preds = load_cup_predictions(root_path + '/results/NN-SGD/CUP/mean_5_internal_test_preds_poly.csv')
nn_adam_internal_test_preds = load_cup_predictions(root_path + '/results/NN-ADAM/CUP/mean_5_internal_test_preds_poly.csv')
svr_internal_test_preds = load_cup_predictions(root_path + '/results/SVM/CUP/internal_test_preds_poly.csv')

For the weighted average we also need to load, for each model, the MEE on the training/validation data.

In [ ]:
def load_cup_train_mee(in_path: str):
    """Utility function to train MEE of a model.
    
    Args:
        - in_dir: the .json file path
    """
    with open(in_path) as inf:
        data = json.load(inf)
        try:
            return data['train']['mean_mee']
        except KeyError:
            return data['train']['mee']

In [ ]:
mee_sgd_train = load_cup_train_mee(root_path + '/results/NN-SGD/CUP/mean_5_report_poly.json')
mee_adam_train = load_cup_train_mee(root_path + '/results/NN-ADAM/CUP/mean_5_report_poly.json')
mee_svr_train = load_cup_train_mee(root_path + '/results/SVM/CUP/report_poly.json')

### Ensembles

In [ ]:
# Arithmetic average
ar_ensemble_train_preds = ensemble_arithmetic_average(svr_trainval_preds, nn_sgd_trainval_preds, nn_adam_trainval_preds)
ar_ensemble_internal_test_preds = ensemble_arithmetic_average(svr_internal_test_preds, nn_sgd_internal_test_preds, nn_adam_internal_test_preds)

print(f"ARITHMETIC AVERAGE TRAIN MEE: {mean_euclidean_error(y_dev_cup_tmp, ar_ensemble_train_preds)}")
print(f"ARITHMETIC AVERAGE INTERNAL TEST MEE: {mean_euclidean_error(y_internal_test_cup, ar_ensemble_internal_test_preds)}")

In [ ]:
# Weighted average (weights w.r.t. lower train MEE)
wa_ensemble_train_preds = ensemble_weighted_averange(
    1, svr_trainval_preds, nn_sgd_trainval_preds, 
    nn_adam_trainval_preds, mee_svr_train, 
    mee_sgd_train, mee_adam_train
)
wa_ensemble_internal_test_preds = ensemble_weighted_averange(
    1, svr_internal_test_preds, nn_sgd_internal_test_preds, 
    nn_adam_internal_test_preds, mee_svr_train, 
    mee_sgd_train, mee_adam_train
)

print(f"WEIGHTED AVERAGE TRAIN MEE: {mean_euclidean_error(y_dev_cup_tmp, wa_ensemble_train_preds)}")
print(f"WEIGHTED AVERAGE INTERNAL TEST MEE: {mean_euclidean_error(y_internal_test_cup, wa_ensemble_internal_test_preds)}")

In [ ]:
# Stacking Ridge
stacking_estimator = Ridge(alpha=30)
x_stack_cup_trainval = np.hstack((svr_trainval_preds, nn_sgd_trainval_preds, nn_adam_trainval_preds))
x_stack_cup_internal_test = np.hstack((svr_internal_test_preds, nn_sgd_internal_test_preds, nn_adam_internal_test_preds))
stacking_estimator.fit(x_stack_cup_trainval, y_dev_cup_tmp)

print(f"STACKING RIDGE TRAIN MEE: {mean_euclidean_error(y_dev_cup_tmp, stacking_estimator_train.predict(x_stack_cup_trainval))}")
print(f"STACKING RIDGE INTERNAL TEST MEE: {mean_euclidean_error(y_internal_test_cup, stacking_estimator.predict(x_stack_cup_internal_test))}")

# Final re-training
Since the test error has already been estimated by leveraging the (untouched) internal test, we now perform a final re-training with all the development data. This does not violate the rules, since the internal test is not (and has never) been used for any model selection.

Thus, we train on the entire development data, i.e. $90$ train/val + $10$ internal test. Early stopping is w.r.t. the MEE.

The best performing model/ensemble - according to the hold-out validation previously performed - is the **Stacking Regressor (Ridge)**. This will be the our final model/ensemble to obtain the blind test set predictions.

In [ ]:
# Apply polynomial to the entire development set
x_dev_cup = poly.transform(x_dev_cup)

### Load blind-test predictions
We load each model blind test prediction after their final re-training (i.e., the re-training on all development data).

In [ ]:
# Load final train predictions
nn_sgd_final_train_preds = load_cup_predictions(root_path + '/results/NN-SGD/CUP/final_train.csv')
nn_adam_final_train_preds = load_cup_predictions(root_path + '/results/NN-ADAM/CUP/final_train.csv')
svr_final_train_preds = load_cup_predictions(root_path + '/results/SVM/CUP/final_train.csv')

# Load final blind test predictions
nn_sgd_blind_test_preds = load_cup_predictions(root_path + '/results/NN-SGD/CUP/final_blind_test.csv')
nn_adam_blind_test_preds = load_cup_predictions(root_path + '/results/NN-ADAM/CUP/final_blind_test.csv')
svr_blind_test_preds = load_cup_predictions(root_path + '/results/SVM/CUP/final_blind_test.csv')

In [ ]:
# Stacking with Ridge on blind test set
stacking_estimator_final = Ridge(alpha=30)
x_stack_cup_final_train = np.hstack((svr_final_train_preds, nn_sgd_final_train_preds, nn_adam_final_train_preds))
x_stack_cup_blind_test = np.hstack((svr_blind_test_preds, nn_sgd_blind_test_preds, nn_adam_blind_test_preds))
stacking_estimator_final.fit(x_stack_cup_final_train, y_dev_cup)

stacking_final_blind_test_preds = stacking_estimator_final.predict(x_stack_cup_blind_test)

In [ ]:
# Store final blind test mean predictions
with open(results_dir + '/CUP/final_blind_test.csv', 'w') as outf:
    # Team Info
    outf.write("# Matteo Pinna, Leonardo Caridi, Marco Sanna\n")
    outf.write("# ACD-TEAM\n")
    outf.write("# ML-CUP23 v2\n")
    outf.write("# 20/01/2024\n")

    # Writing predictions
    for i, pred in enumerate(stacking_final_blind_test_preds, 1):
        outf.write(f"{i},{','.join(map(str, pred))}\n")